In [6]:
from functools import partial
from sqlite3 import connect

import astroplan as ap
import astropy.units as u
from icecream import ic, install

from astropaul.database import database_connection
import astropaul.html as html
import astropaul.phase as ph
import astropaul.priority as pr
import astropaul.targetlistcreator as tlc

install() # icecream will now work in imported packages

%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
session = tlc.ObservingSession(ap.Observer.at_site("APO"))
session.add_full_day("2025-08-08")
session.add_full_day("2025-08-09")
session.add_full_day("2025-08-10")
session.add_full_day("2025-08-12")
session.add_full_day("2025-08-13")

synthetic_phase_percent = 0.02

phase_event_defs = [
    ph.PhaseEventDef("Mid Eclipse", ph.calc_mid_eclipse),
    ph.PhaseEventDef("Eclipse", ph.calc_mid_eclipse),
    ph.PhaseEventDef(
        "Not in Eclipse", partial(ph.calc_time_of_gress, synthetic_phase_percent=synthetic_phase_percent, ingress=False)
    ),
    ph.PhaseEventDef("Eclipse", partial(ph.calc_time_of_gress, synthetic_phase_percent=synthetic_phase_percent, ingress=True)),
]

min_altitude = 35 * u.deg
max_magnitude = 13

with database_connection() as conn:

    name = "APO Observing List 2025-08-08 to -13"
    creator = tlc.TargetListCreator(name=name, connection=conn)
    creator.steps = [
        tlc.add_targets,
        partial(tlc.filter_targets, criteria=lambda df: df["Target Type"].isin(["QuadEB", "SextEB"])),
        tlc.ancillary_data_from_tess,
        partial(tlc.filter_targets, criteria=lambda df: df["Vmag"] < max_magnitude),
        partial(
            tlc.add_observability,
            observing_session=session,
            calc_moon_distance=True,
            observability_threshold=(min_altitude, 80 * u.deg),
        ),
        partial(tlc.filter_targets, criteria=lambda df: df["Observable Any Night"]),
        partial(tlc.add_database_table, table_name="ephemerides"),
        partial(tlc.add_database_table, table_name="dssi_observations"),
        partial(
            tlc.add_system_configuration, table_name="DSSI Observations", time_column="Mid JD", eclipse_table="SIDE Observations"
        ),
        partial(tlc.add_database_table, table_name="speckle_detections"),
        partial(tlc.filter_targets, criteria=lambda df: df["Num Speckle Detections"] > 0),
        # partial(tlc.filter_other_list, list_name="Speckle Detections", criteria=lambda df: df["Separation"] < 0.1 * u.arcsec),
        partial(
            tlc.add_phase_events,
            observing_session=session,
            phase_event_defs=phase_event_defs,
            event_types=["Mid Eclipse", "Eclipse"],
        ),
    ]
    tl = creator.calculate(verbose=False)

    print(tl.summarize())
    # tl.target_list

Name: APO Observing List 2025-08-08 to -13
Criteria
    lambda df: df["Target Type"].isin(["QuadEB", "SextEB"])
    lambda df: df["Vmag"] < 13
    AltitudeConstraint: {'min': 35.0, 'max': 80.0, 'boolean_constraint': True}
    lambda df: df["Observable Any Night"]
    lambda df: df["Num Speckle Detections"] > 0
24 targets:
      24 QuadEB
Column Count (primary, secondary):
    Target: (3, 4)
    TESS Data: (2, 4)
    RV Data: (1, 2)
    Observable: (5, 20)
    Count: (4, 0)
Associated tables:
       5 rows,  2 columns: Lunar Phases
     181 rows,  7 columns: Ephemerides
     130 rows, 12 columns: DSSI Observations
      45 rows,  6 columns: SIDE Observations
      78 rows, 11 columns: Speckle Detections
     494 rows, 10 columns: Phase Events



In [11]:
altitude_categories = [
    ((-90, min_altitude.value), 0),
    ((min_altitude.value, 90), 1),
]

pl = pr.PriorityList(tl, session, interval=30 * u.min)
pr.calculate_altitude_priority(pl, altitude_categories=altitude_categories)
# pr.prioritize_phase_sequence(pl, ["Eclipse", "Mid Eclipse", "Eclipse"], "Eclipse", False, False, entirely_observable=True)
pr.prioritize_phase_sequence(pl, ["Eclipse"], "Eclipse", True, True, entirely_observable=False)
pr.prioritize_side_observation(pl, side_state="Eclipse")
pr.calculate_overall_priority(pl)
pr.aggregate_target_priorities(pl, skip_column_threshold=0.1)
pl.categorize_priorities(bins=[0.00, 0.20, 0.40, 0.6, 1.00], labels=["", "*", "* *", "* * *"])

html.render_observing_pages(tl, pl, {}, "../../../../Observing Files/APO Observing 2025-08-08")
# pl.numerical_priorities[0]